# Data Cleaning

## Introduction

* This notebook goes through a necasssary step of cleaning the data before it is used for exploratory data analysis. 
* The input of this notebook is a training dataset in csv format sourced from Kaggle. 
* The output of this notebook is a Term frequency-inverse document frequency(TF-IDF) Matrix

## Reading and Understanding the dataset

In [343]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # regular expression 
import string       # String Handling
import random       #For selecting random rows
import nltk         # Natural langauag processing toolkit

Reading the training data into a dataframe using pandas and viewing the top 20 rows of data

In [324]:
train_df = pd.read_csv("train.csv")
train_df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


First 20 rows in 'Keyword' and 'Location' column are NaNs. Now Checking if there are nulls in other columns of the dataframe.

In [325]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Checking number of unique values in 'keywords' and 'location'.

In [326]:
len(train_df.keyword.unique())

222

In [327]:
len(train_df.location.unique())

3342

A glimpse at 50 rows from the 'locations' column selected randomly


In [328]:
random.sample(list((train_df.location.unique())),k=50)


['Galveston, Texas',
 'Gaborone, Botswana',
 'Methville, CA',
 'Leicester, England',
 'Epic City, BB.',
 'LA/OC/Vegas',
 'New Delhi, Delhi',
 'taking pain like pleasure',
 'Silver Spring, MD',
 'Youngstown, OH',
 'Stay Tuned ;) ',
 'EARTH',
 'ny',
 'Victoria, Tx.',
 'Horsemind, MI',
 'RSN: Tru',
 'Unites States',
 'Fort Collins, CO',
 'ELVY',
 'El Dorado, Arkansas',
 'STL ?NOLA',
 'CA physically- Boston Strong?',
 'Jump City',
 'Dudetown',
 'Kawartha Lakes, Ontario, Canad',
 'EveryWhere',
 'KOLKATA',
 'Kokomo, In',
 'å_å_Los Mina City\x89ã¢',
 'Santiago Bernabeau',
 'contactSimpleNews@gmail.com',
 'Studio',
 'NYC area',
 'The P (South Philly)',
 'chillin at ceder rapids',
 'Wausau, Wisconsin',
 'Bristol, England',
 'Bishops Lydeard, England',
 'Between the worlds ',
 'Moscow',
 'New Chicago',
 'Atlanta(ish), GA',
 "elena's bed // info on link",
 'Insula Barataria',
 'Alger-New York-San Francisco',
 'Whiterun, Skyrim',
 '//RP\\ ot @Mort3mer\\\\',
 'Miami Beach, Fl',
 'Fruit Bowl',
 'Sun

A glimpse at 50 rows from the 'keywords' column selected randomly

In [329]:
random.sample(list((train_df.keyword.unique())),k=50)


['fatality',
 'cliff%20fall',
 'destruction',
 'obliterated',
 'demolition',
 'death',
 'bush%20fires',
 'crashed',
 'flames',
 'meltdown',
 'dead',
 'evacuation',
 'detonation',
 'bleeding',
 'crushed',
 'sirens',
 'fire',
 'harm',
 'bioterror',
 'blaze',
 'engulfed',
 'siren',
 'burned',
 'suicide%20bomber',
 'violent%20storm',
 'bridge%20collapse',
 'forest%20fires',
 'drowned',
 'devastation',
 'inundation',
 'injured',
 'arson',
 'exploded',
 'blood',
 'typhoon',
 'hijack',
 'hellfire',
 'wounds',
 'aftershock',
 'hostage',
 'deluge',
 'annihilated',
 'seismic',
 'nuclear%20reactor',
 'traumatised',
 'emergency%20services',
 'survived',
 'threat',
 'demolished',
 'burning%20buildings']

A glimpse at 30 rows from the 'text' column selected randomly

In [330]:
# Selecting 30 random from text column randomly
random.sample(list((train_df.text)),k=30)

["@Dead_Dreamer15 ...because if it were on fire that'd be a safety hazard",
 'why are people panicking about results day though hahahah like worrying is going to change your results',
 'Twister was fun https://t.co/qCT6fb8wOn',
 'Maryland mansion fire that killed 6 caused by damaged plug under Christmas tree report says - Into the flames... http://t.co/P1WmkXA9d8',
 '@YoungHeroesID 4. Lava Blast Power Red #PantherAttack',
 '4 THOSE WHO CARE ABOUT SIBLING ABUSE SURVIVORS join the new family tree: http://t.co/LQD1WEfpQd http://t.co/GgnbVZoHWu',
 "@TheLegendBlue @Cozmo23 they'll probably allow us to ascend them but not get them to the damage max values",
 "@SoDamnTrue  we know who u are you're a bleeding heart wannabe pickup artist",
 'im feeling attacked http://t.co/91jvYCxXVi',
 '@_STiiiLO I still got video of u demolished',
 "The way you move is like a full on rainstorm and I'm a house of cards.",
 'Coastal German Shepherd Rescue OC shared a link... http://t.co/P85NwcMkQu #animalrescue

## Observation

Some data incosistencies or redundant information found in the dataset are as follows

* Upper case and lower case at unexpected location
* Punctuations
* Numbers in text 
* Use of cities, states and Country names. (Granularity problem)
* Special characters such as \x89ÛÒ and \n
* Hyperklinks
* Tags in tweets




## Cleaning the data

The method below does the following to clean anomalies in 'location' column:

* Changes all the text to lower case
* Removes punctutations
* Removes texts with numbers
* Removes cities names if country/state names are mentioned. (High level granularity is maintained)

In [331]:
# Method to chaange text to lower case and remove punctionation
def cleaning_location(text):
    text = str(text)
    text = text.lower()    #lower case
    text = text.split(',')[-1:][0].strip() # Removing city names when country/state name is present
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #removing punctuations
    text = re.sub('\w*\d\w*', '', text) #removing text with number

    
    return text

# Applying the method to location column
train_df.location = train_df.location.apply(cleaning_location)

The method below does the following to clean anomalies in 'keyword' column:

* Changes all the text to lower case
* Removes punctutations
* replaces number with space (as %20 was found in middle of two words)

In [332]:
train_df.location

0       nan
1       nan
2       nan
3       nan
4       nan
       ... 
7608    nan
7609    nan
7610    nan
7611    nan
7612    nan
Name: location, Length: 7613, dtype: object

In [333]:
def cleaning_keyword(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('(\d+)', ' ', text)   #replacing numbers with space
    return text

# Applying the method to keyword column
train_df.keyword = train_df.keyword.apply(cleaning_keyword)

The following method to clean anomalies in 'text' column does the following

* Changes all the text to lower case
* Removes words starting with @ to remove the tags and mentions example: @barackobama
* Removes links
* Removes punctuation
* Removes words with numbers
* Removes special characters examples: \x89û,\x89ûó etc 
* Removes '\n' 


In [334]:
def cleaning_text(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub(r'@\w+', '',text) ##removing any word starting with @
    text = re.sub(r'https|www|http\S+', '', text)  ##removing any word starting with http
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('\w*\d\w*', '', text)  #removing words with numbers
    text = re.sub(r'[^\x00-\x7F]+', '', text) # removing special characters
    text.replace("\n","")
    return text 

# Applying the method to the text column in the dataframe
train_df.text = train_df.text.apply(cleaning_text)

A glimpse of cleaned data set

In [340]:
train_df.sample(n=30)

,id,keyword,location,text,target
2452,3520,derailment,nan,madhya pradesh train derailment village youth ...,1
1562,2256,cliff fall,neverland,fuck neil go fall off a cliff or somethingyr,0
4960,7070,meltdown,nan,why must i have a meltdown every few days,0
829,1206,blizzard,usa,not really that shocking blizzard lured thei...,0
1293,1866,burned,nan,i put my chicken nuggets in the microwave for ...,0
5812,8295,rubble,nan,if you go on with this nuclear arms race all y...,0
5292,7559,outbreak,nan,families to sue over legionnaires more than f...,1
3644,5193,fatalities,oman muscat al seeb,i liked a video tremor is no joke tremor dlc...,1
4580,6513,injuries,nan,right yep youre a witness to his injuries ha...,0
2695,3866,detonation,nan,ignition knock detonation sensorsenso beckarnl...,1


Requirement already up-to-date: nltk in /Users/shashanksharma/opt/anaconda3/lib/python3.7/site-packages (3.4.5)
Note: you may need to restart the kernel to use updated packages.
